In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [2]:
# saw this one in a video and tought this was a good idea to improve 
# the model, eventought we lose some information
# but for this kind of data is worthy

red_wine = 'C://Users//vinic//Documents//Python//Machine Learning//Projetos UCI ML//Wine Quality//wine+quality//winequality-red.csv'
white_wine = 'C://Users//vinic//Documents//Python//Machine Learning//Projetos UCI ML//Wine Quality//wine+quality//winequality-white.csv'
data_red_wine = pd.read_csv(red_wine,header=0,sep=';')
data_white_wine = pd.read_csv(white_wine,header=0,sep=';')


In [3]:
bins = [0,5.5,7.5,10]
labels = [0,1,2]

data_red_wine['quality'] = pd.cut(data_red_wine['quality'],bins = bins, labels = labels)

data_white_wine['quality'] = pd.cut(data_white_wine['quality'],bins = bins, labels = labels)


In [4]:
data_white_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,1
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,1
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,1


## Train, Validation and test Data Sets

In [5]:
#data.sample shuffles the data

red_wine_train, red_wine_valid, red_wine_test = np.split(data_red_wine.sample(frac=1), [int(0.6*len(data_red_wine)), int(0.8*len(data_red_wine))])
white_wine_train, white_wine_valid, white_wine_test = np.split(data_white_wine.sample(frac=1), [int(0.6*len(data_white_wine)), int(0.8*len(data_white_wine))])

    


c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
def scale_dataset(dataframe,oversample = False):
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values
    
    scaler = StandardScaler()
    #Scale the number, so we dont have  huge discrepancy between columns, it affects the model
    X = scaler.fit_transform(X)
    
    #the difference between the len of the values must not be huge, so we have to scale it
    #oversample the one that has the least, taking more of the less class
    
    if oversample:
        ros = RandomOverSampler()
        X,y = ros.fit_resample(X,y)
        
    # concat 2 arrays, y is only one dimension so we have to make it 2 
    #in this function using -1 is the same as len(y)
    #same as concat in pandas put hstack in numpay
    df = np.hstack((X,np.reshape(y,(-1,1))))
    
    return df,X,y


In [7]:
## red
red_wine_train, red_wine_X_train, red_wine_y_train = scale_dataset(red_wine_train,oversample=True)
red_wine_valid, red_wine_X_valid, red_wine_y_valid = scale_dataset(red_wine_valid,oversample=False)
red_wine_test, red_wine_X_test, red_wine_y_test = scale_dataset(red_wine_test,oversample=False)

## white

white_wine_train, white_wine_X_train, white_wine_y_train = scale_dataset(white_wine_train,oversample=True)
white_wine_valid, white_wine_X_valid, white_wine_y_valid = scale_dataset(white_wine_valid,oversample=False)
white_wine_test, white_wine_X_test, white_wine_y_test = scale_dataset(white_wine_test,oversample=False)



## Predict the Quality in the margins

# K-nearest neighbors - KNN

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [9]:
## red wine
red_wine_knn_model = KNeighborsClassifier(n_neighbors=1)
red_wine_knn_model.fit(red_wine_X_train,red_wine_y_train)
#use the knn model to try to predict the test dataset
red_wine_y_pred = red_wine_knn_model.predict(red_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine Knn\n',classification_report(red_wine_y_test,red_wine_y_pred))

For the Red Wine Knn
               precision    recall  f1-score   support

           0       0.69      0.76      0.73       140
           1       0.78      0.71      0.74       176
           2       0.25      0.25      0.25         4

    accuracy                           0.73       320
   macro avg       0.57      0.57      0.57       320
weighted avg       0.73      0.73      0.73       320



In [10]:
## improved 0.74 - 0.57 = 0.17 , in comparisson to using all the qualities

In [11]:
## white wine
white_wine_knn_model = KNeighborsClassifier(n_neighbors=1)
white_wine_knn_model.fit(white_wine_X_train,white_wine_y_train)
#use the knn model to try to predict the test dataset
white_wine_y_pred = white_wine_knn_model.predict(white_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the White Wine Knn\n',classification_report(white_wine_y_test,white_wine_y_pred,zero_division=True))

For the White Wine Knn
               precision    recall  f1-score   support

           0       0.67      0.65      0.66       329
           1       0.79      0.80      0.80       619
           2       0.43      0.41      0.42        32

    accuracy                           0.74       980
   macro avg       0.63      0.62      0.63       980
weighted avg       0.74      0.74      0.74       980



In [12]:
## improved 0.73 - 0.56 = 0.17 , in comparisson to using all the qualities

# Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB

In [14]:
## red wine
red_wine_nb_model = GaussianNB()
red_wine_nb_model.fit(red_wine_X_train,red_wine_y_train)
#use the knn model to try to predict the test dataset
red_wine_y_pred = red_wine_nb_model.predict(red_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine NB\n',classification_report(red_wine_y_test,red_wine_y_pred))

For the Red Wine NB
               precision    recall  f1-score   support

           0       0.72      0.76      0.74       140
           1       0.72      0.49      0.58       176
           2       0.02      0.25      0.04         4

    accuracy                           0.60       320
   macro avg       0.49      0.50      0.45       320
weighted avg       0.71      0.60      0.64       320



In [15]:
## improved 0.53 - 0.41 = 0.12 , in comparisson to using all the qualities

In [16]:
## white wine
white_wine_nb_model = GaussianNB()
white_wine_nb_model.fit(white_wine_X_train,white_wine_y_train)

white_wine_y_pred = white_wine_nb_model.predict(white_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the White Wine NB\n',classification_report(white_wine_y_test,white_wine_y_pred,zero_division=True))

For the White Wine NB
               precision    recall  f1-score   support

           0       0.56      0.73      0.64       329
           1       0.73      0.18      0.29       619
           2       0.06      0.69      0.10        32

    accuracy                           0.38       980
   macro avg       0.45      0.53      0.34       980
weighted avg       0.65      0.38      0.40       980



In [17]:
## improved 0.40 - 0.47 = -0.07 , in comparisson to using all the qualities

In [18]:
# SVM

In [19]:
from sklearn.svm import SVC

In [20]:
## red wine
red_wine_svm_model = SVC()
red_wine_svm_model.fit(red_wine_X_train,red_wine_y_train)
#use the knn model to try to predict the test dataset
red_wine_y_pred = red_wine_svm_model.predict(red_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine SVM\n',classification_report(red_wine_y_test,red_wine_y_pred))

For the Red Wine SVM
               precision    recall  f1-score   support

           0       0.69      0.86      0.77       140
           1       0.84      0.64      0.73       176
           2       0.08      0.25      0.12         4

    accuracy                           0.73       320
   macro avg       0.54      0.58      0.54       320
weighted avg       0.76      0.73      0.74       320



In [21]:
## improved 0.70 - 0.46 = 0.24 , in comparisson to using all the qualities

In [22]:
## white wine
white_wine_svm_model = SVC()
white_wine_svm_model.fit(white_wine_X_train,white_wine_y_train)

white_wine_y_pred = white_wine_svm_model.predict(white_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the White Wine SVM\n',classification_report(white_wine_y_test,white_wine_y_pred,zero_division=True))

For the White Wine SVM
               precision    recall  f1-score   support

           0       0.60      0.76      0.67       329
           1       0.81      0.55      0.66       619
           2       0.13      0.59      0.21        32

    accuracy                           0.62       980
   macro avg       0.52      0.64      0.52       980
weighted avg       0.72      0.62      0.65       980



In [23]:
## improved 0.61 - 0.45 = 0.24 , in comparisson to using all the qualities

# Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
## red wine
red_wine_lr_model = LogisticRegression()
red_wine_lr_model.fit(red_wine_X_train,red_wine_y_train)
#use the knn model to try to predict the test dataset
red_wine_y_pred = red_wine_lr_model.predict(red_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine LR\n',classification_report(red_wine_y_test,red_wine_y_pred))

For the Red Wine LR
               precision    recall  f1-score   support

           0       0.70      0.84      0.77       140
           1       0.79      0.52      0.63       176
           2       0.03      0.25      0.05         4

    accuracy                           0.66       320
   macro avg       0.51      0.54      0.48       320
weighted avg       0.74      0.66      0.68       320



In [26]:
## improved 0.65 - 0.39 = 0.26 , in comparisson to using all the qualities

In [27]:
## white wine
white_wine_lr_model = LogisticRegression()
white_wine_lr_model.fit(white_wine_X_train,white_wine_y_train)

white_wine_y_pred = white_wine_lr_model.predict(white_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the White Wine LR\n',classification_report(white_wine_y_test,white_wine_y_pred,zero_division=True))

For the White Wine LR
               precision    recall  f1-score   support

           0       0.56      0.75      0.64       329
           1       0.76      0.30      0.43       619
           2       0.07      0.62      0.12        32

    accuracy                           0.46       980
   macro avg       0.46      0.56      0.40       980
weighted avg       0.67      0.46      0.49       980



In [28]:
## improved 0.51 - 0.49 = 0.02 , in comparisson to using all the qualities

# Decision Tree Classification

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [30]:
## red wine
red_wine_dtc_model = DecisionTreeClassifier()
red_wine_dtc_model.fit(red_wine_X_train,red_wine_y_train)
#use the knn model to try to predict the test dataset
red_wine_y_pred = red_wine_dtc_model.predict(red_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine DTC\n',classification_report(red_wine_y_test,red_wine_y_pred))

For the Red Wine DTC
               precision    recall  f1-score   support

           0       0.65      0.79      0.72       140
           1       0.78      0.64      0.70       176
           2       0.00      0.00      0.00         4

    accuracy                           0.70       320
   macro avg       0.48      0.48      0.47       320
weighted avg       0.71      0.70      0.70       320



In [31]:
## improved 0.72 - 0.51 = 0.21 , in comparisson to using all the qualities

In [32]:
## white wine
white_wine_dtc_model = DecisionTreeClassifier()
white_wine_dtc_model.fit(white_wine_X_train,white_wine_y_train)

white_wine_y_pred = white_wine_dtc_model.predict(white_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the White Wine dtc\n',classification_report(white_wine_y_test,white_wine_y_pred,zero_division=True))

For the White Wine dtc
               precision    recall  f1-score   support

           0       0.59      0.61      0.60       329
           1       0.76      0.74      0.75       619
           2       0.31      0.38      0.34        32

    accuracy                           0.68       980
   macro avg       0.55      0.57      0.56       980
weighted avg       0.69      0.68      0.69       980



In [33]:
## improved 0.72 - 0.59 = 0.13 , in comparisson to using all the qualities

# Random Forest Classification

In [50]:
## red wine
red_wine_rfc_model = RandomForestClassifier()
red_wine_rfc_model.fit(red_wine_X_train,red_wine_y_train)
#use the knn model to try to predict the test dataset
red_wine_y_pred = red_wine_rfc_model.predict(red_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine rfc\n',classification_report(red_wine_y_test,red_wine_y_pred))

For the Red Wine rfc
               precision    recall  f1-score   support

           0       0.74      0.84      0.78       140
           1       0.83      0.76      0.80       176
           2       0.00      0.00      0.00         4

    accuracy                           0.78       320
   macro avg       0.52      0.53      0.53       320
weighted avg       0.78      0.78      0.78       320



c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
## improved 0.70 - 0.50 = 0.20 , in comparisson to using all the qualities

In [36]:
## white wine
white_wine_rfc_model = RandomForestClassifier()
white_wine_rfc_model.fit(white_wine_X_train,white_wine_y_train)

white_wine_y_pred = white_wine_rfc_model.predict(white_wine_X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the White Wine rfc\n',classification_report(white_wine_y_test,white_wine_y_pred,zero_division=True))

For the White Wine rfc
               precision    recall  f1-score   support

           0       0.74      0.72      0.73       329
           1       0.82      0.86      0.84       619
           2       0.79      0.34      0.48        32

    accuracy                           0.79       980
   macro avg       0.78      0.64      0.68       980
weighted avg       0.79      0.79      0.79       980



In [37]:
## improved 0.80 - 0.66 = 0.14 , in comparisson to using all the qualities

## Neural Network

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Input


# Red Whine NN

In [39]:
def train_model_red(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs,X_valid,y_valid):
    col = X_train.shape[1]
    num_classes = len(np.unique(y_train))
    nn_model = tf.keras.Sequential(
        [Input(shape=(col,)),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(num_classes,activation='softmax')
        ]
    )
    
    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    history = nn_model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid,y_valid),verbose=0
        )
    return nn_model, history


In [40]:
import numpy as np
import tensorflow as tf
from itertools import product
from sklearn.metrics import f1_score

def find_best_param(X_train, y_train, X_valid, y_valid):
    # Seed para reprodutibilidade
    tf.random.set_seed(42)
    np.random.seed(42)

    # Parâmetros para busca
    num_nodes_list = [16, 32, 34]
    dropout_probs = [0, 0.2]
    learning_rates = [0.01, 0.005, 0.001]
    batch_sizes = [32, 64, 128]
    epochs = 100

    # Inicialização do melhor modelo
    best_model_params = None
    best_f1 = 0
    best_accuracy = 0
    best_val_loss = float('inf')

    # Loop pelos hiperparâmetros
    for num_nodes, dropout_prob, lr, batch_size in product(num_nodes_list, dropout_probs, learning_rates, batch_sizes):
        print(f"\nTestando: nodes={num_nodes}, dropout={dropout_prob}, lr={lr}, batch={batch_size}")

        # Treina o modelo com os parâmetros atuais
        model, history = train_model_red(
            X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs, X_valid, y_valid
        )

        # Avaliação no conjunto de validação
        val_loss = history.history['val_loss'][-1]
        val_accuracy = history.history['val_accuracy'][-1]

        y_val_pred_probs = model.predict(X_valid)
        y_val_pred = np.argmax(y_val_pred_probs, axis=1)  # <- Aqui é a correção para multi-classe
        val_f1 = f1_score(y_valid, y_val_pred, average='weighted')

        # Mostra métricas da iteração
        print(f"F1: {val_f1:.4f}, Acc: {val_accuracy:.4f}, Loss: {val_loss:.4f}")

        # Atualiza o melhor conjunto de parâmetros
        if (val_f1 > best_f1) or (val_f1 == best_f1 and val_accuracy > best_accuracy) or (val_f1 == best_f1 and val_loss < best_val_loss):
            best_f1 = val_f1
            best_accuracy = val_accuracy
            best_val_loss = val_loss
            best_model_params = {
                "num_nodes": num_nodes,
                "dropout_prob": dropout_prob,
                "lr": lr,
                "batch_size": batch_size
            }

    return best_model_params


In [41]:
# red wine
best_model_params = find_best_param(red_wine_X_train, red_wine_y_train,red_wine_X_valid,red_wine_y_valid)



Testando: nodes=16, dropout=0, lr=0.01, batch=32
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  
F1: 0.7069, Acc: 0.7125, Loss: 1.8085

Testando: nodes=16, dropout=0, lr=0.01, batch=64
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.6818, Acc: 0.6875, Loss: 1.8949

Testando: nodes=16, dropout=0, lr=0.01, batch=128
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.6667, Acc: 0.6687, Loss: 1.9277

Testando: nodes=16, dropout=0, lr=0.005, batch=32
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.7062, Acc: 0.7063, Loss: 1.5276

Testando: nodes=16, dropout=0, lr=0.005, batch=64
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.7173, Acc: 0.7219, Loss: 1.4442

Testando: nodes=16, dropout=0, lr=0.005, batch=128
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.7138, Acc: 0.7188, Loss: 1.0568

Testando: nodes=16, dropout=0, lr=0.001, batch=32
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.6984, Acc: 0.7000, Loss: 0.8289

Testando: nodes=16, dropout=0, lr=0.001, batch=64
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
F1: 0.

In [43]:
epochs = 100

In [44]:
print(best_model_params)
model,history = train_model_red(red_wine_X_train, red_wine_y_train, best_model_params['num_nodes'], best_model_params['dropout_prob'], best_model_params['lr'], best_model_params['batch_size'], epochs,red_wine_X_valid,red_wine_y_valid)

{'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 32}


In [45]:
import numpy as np
from sklearn.metrics import classification_report


# Realizar previsões no conjunto de teste
y_pred_probs = model.predict(red_wine_X_test)

# Obter a classe com maior probabilidade
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Avaliar o modelo
print(classification_report(red_wine_y_test, y_pred_classes))

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.69      0.80      0.74       140
           1       0.80      0.69      0.74       176
           2       0.25      0.25      0.25         4

    accuracy                           0.73       320
   macro avg       0.58      0.58      0.58       320
weighted avg       0.74      0.73      0.73       320



In [46]:
## improved 0.73 - 0.47 = 0.26 , in comparisson to using all the qualities

# White Wine NN

In [47]:
def train_model_white(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs,X_valid,y_valid):
    col = X_train.shape[1]
    num_classes = len(np.unique(y_train))
    nn_model = tf.keras.Sequential(
        [Input(shape=(col,)),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(num_classes,activation='softmax')
        ]
    )
    
    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    history = nn_model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid,y_valid),verbose=0
        )
    return nn_model, history


In [48]:
# white wine


best_model_params = find_best_param(white_wine_X_train, white_wine_y_train,white_wine_X_valid,white_wine_y_valid)


Testando: nodes=16, dropout=0, lr=0.01, batch=32
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F1: 0.6648, Acc: 0.6571, Loss: 1.2768

Testando: nodes=16, dropout=0, lr=0.01, batch=64
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F1: 0.6846, Acc: 0.6735, Loss: 1.2729

Testando: nodes=16, dropout=0, lr=0.01, batch=128
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F1: 0.6840, Acc: 0.6765, Loss: 1.2308

Testando: nodes=16, dropout=0, lr=0.005, batch=32
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
F1: 0.6942, Acc: 0.6867, Loss: 1.1980

Testando: nodes=16, dropout=0, lr=0.005, batch=64
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F1: 0.6766, Acc: 0.6694, Loss: 1.0464

Testando: nodes=16, dropout=0, lr=0.005, batch=128
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F1: 0.7013, Acc: 0.6939, Loss: 0.8284

Testando: nodes=16, dropout=0, lr=0.001, batch=32
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
F1: 0.6679, Acc: 0.6541, Loss: 0.8300

Testando: nodes=16, dropout=0, lr=0.001, batch=64
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
F1: 0.6871, Acc

In [55]:
print(best_model_params)

model,history = train_model_white(white_wine_X_train, white_wine_y_train, best_model_params['num_nodes'], best_model_params['dropout_prob'], best_model_params['lr'], best_model_params['batch_size'], epochs,white_wine_X_valid,white_wine_y_valid)

{'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 64}


In [56]:
import numpy as np
from sklearn.metrics import classification_report

# Realizar previsões no conjunto de teste
y_pred_probs = model.predict(white_wine_X_test)

# Obter a classe com maior probabilidade
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Avaliar o modelo
print(classification_report(white_wine_y_test, y_pred_classes,zero_division=True))


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

           0       0.61      0.67      0.64       329
           1       0.78      0.73      0.75       619
           2       0.31      0.41      0.35        32

    accuracy                           0.70       980
   macro avg       0.57      0.60      0.58       980
weighted avg       0.71      0.70      0.70       980



In [ ]:
## improved 0.61 - 0.46 = 0.26 , in comparisson to using all the qualities